### This jupyter notebook contains the algorithmic part of the proof that the _Noperthedron_ is not Rupert.

The notebook consists of three parts:
1) [**General solution-tree integrety**](#General-solution-tree-integrety): We make sure, that the tree does not have any consistency faults. More precisely, we check:
   - the IDs are unique numbers and there are no gaps,
   - there are only 3 possible node types (1,2 or 3),
   - the root of the tree contains the desired interval,
   - the union of the regions of every child (row with nodeType=1) is indeed the region of their parent.
3) [**Checking global Theorems**](#Checking-the-global-Theorem): We check that the rational global theorem can be applied to each node with nodeType=2.

5) [**Checking local Theorems**](#Checking-the-local-Theorem): We check that the rational local theorem can be applied to each node with nodeType=3.

In [ ]:
from helper_functions import * # import all kinds of useful helper functions
from noperthedron import NOP # import the Noperthedron as NOP 

import pandas as pd # to read the tabular solution tree

from tqdm import tqdm

INTERVAL_DENOMINATOR = 15360000
R_DENOMINATOR = 1000

kappa=10**(-10)

TREE = pd.read_parquet("../data/solution_tree.parquet") # load the solution tree
#can also load TREE from the csv with next line. For that first unzip the zip file ../data/solution_tree.zip. 
#Caution: loading from csv takes quite long! 
#from load_tree_from_csv import TREE

RatNOP = matrix(QQ,90,3) # define the rational approximation of the Noperthedron
for row in range(NOP.nrows()):
    for col in range(NOP.ncols()):
        RatNOP[row,col]=floor(NOP[row,col]*10**16)/10**16

In [ ]:
print("The first three vertices of the Noperthedron are:")
pretty_print(NOP[:3]) 

print("And the rational approximation of them is:")
pretty_print(RatNOP[:3])

In [ ]:
print("The solution tree looks like this:")
TREE

# General solution-tree integrety 

In [ ]:
row=TREE.loc[int(0)]
assert row["T1_min"]==INTERVAL_DENOMINATOR*0
assert row["T1_max"]==INTERVAL_DENOMINATOR*42/100
assert row["V1_min"]==INTERVAL_DENOMINATOR*0
assert row["V1_max"]==INTERVAL_DENOMINATOR*315/100
assert row["T2_min"]==INTERVAL_DENOMINATOR*0
assert row["T2_max"]==INTERVAL_DENOMINATOR*42/100
assert row["V2_min"]==INTERVAL_DENOMINATOR*0
assert row["V2_max"]==INTERVAL_DENOMINATOR*158/100
assert row["A_min"] ==INTERVAL_DENOMINATOR*(-158/100)
assert row["A_max"] ==INTERVAL_DENOMINATOR*158/100
print(f"Test passed! The root represents the correct interval:")
print(f"    The root represents the interval theta1 = [{row['T1_min']/INTERVAL_DENOMINATOR} , {row['T1_max']/INTERVAL_DENOMINATOR}]")
print(f"    The root represents the interval phi1   = [{row['V1_min']/INTERVAL_DENOMINATOR} , {row['V1_max']/INTERVAL_DENOMINATOR}]")
print(f"    The root represents the interval theta2 = [{row['T2_min']/INTERVAL_DENOMINATOR} , {row['T2_max']/INTERVAL_DENOMINATOR}]")
print(f"    The root represents the interval phi2   = [{row['V2_min']/INTERVAL_DENOMINATOR} , {row['V2_max']/INTERVAL_DENOMINATOR}]")
print(f"    The root represents the interval alpha  = [{row['A_min'] /INTERVAL_DENOMINATOR} , {row['A_max'] /INTERVAL_DENOMINATOR}]")


In [ ]:
column_ID = TREE["ID"]
assert list(column_ID) == list(range(len(column_ID))), "The IDs of the nodes are not numbered sequentially!"
print("Test passed! The IDs of the nodes are numbered sequentially.")

In [ ]:
nrGlobal       = (TREE["nodeType"] == 1).sum()
nrLocal     = (TREE["nodeType"] == 2).sum()
nrInnerNodes = (TREE["nodeType"] == 3).sum()

assert nrGlobal+nrLocal+nrInnerNodes==TREE.shape[0]

print("Test passed! There are only 3 nodetypes:")
print(f"    There are {nrGlobal} nodes with nodetype {1}, i.e. global Theorem applications")
print(f"    There are {nrLocal} nodes with nodetype {2}, i.e. local Theorem applications")
print(f"    There are {nrInnerNodes} nodes with nodetype {3}, i.e. split into subintervals")

In [ ]:
for i in tqdm(range(len(TREE))):
    
    nodetype=TREE["nodeType"].iloc[int(i)]

    if nodetype!=3: # only look at inner nodes
        continue
    
    nrChildren=TREE["nrChildren"].iloc[int(i)]
    split=TREE["split"].iloc[int(i)]
    IDfirstChild=TREE["IDfirstChild"].iloc[int(i)]
    
    T1_min=TREE["T1_min"].iloc[int(i)]
    T1_max=TREE["T1_max"].iloc[int(i)]
    V1_min=TREE["V1_min"].iloc[int(i)]
    V1_max=TREE["V1_max"].iloc[int(i)]
    T2_min=TREE["T2_min"].iloc[int(i)]
    T2_max=TREE["T2_max"].iloc[int(i)]
    V2_min=TREE["V2_min"].iloc[int(i)]
    V2_max=TREE["V2_max"].iloc[int(i)]
    A_min =TREE["A_min"].iloc[int(i)]
    A_max =TREE["A_max"].iloc[int(i)]
    
    if split<=5: 
        ## only one of the intervals will be split
        ## split==1 -> only theta1 will be split
        ## split==2 -> only phi1   will be split
        ## split==3 -> only theta2 will be split
        ## split==4 -> only phi2   will be split
        ## split==5 -> only alpha  will be split

        for childNr in range(nrChildren):
            childID=IDfirstChild+childNr
            
            assert split==1 or TREE["T1_min"].iloc[int(childID)]==T1_min
            assert split==1 or TREE["T1_max"].iloc[int(childID)]==T1_max
            assert split==2 or TREE["V1_min"].iloc[int(childID)]==V1_min
            assert split==2 or TREE["V1_max"].iloc[int(childID)]==V1_max
            assert split==3 or TREE["T2_min"].iloc[int(childID)]==T2_min
            assert split==3 or TREE["T2_max"].iloc[int(childID)]==T2_max
            assert split==4 or TREE["V2_min"].iloc[int(childID)]==V2_min
            assert split==4 or TREE["V2_max"].iloc[int(childID)]==V2_max
            assert split==5 or TREE["A_min"].iloc[int(childID)] ==A_min
            assert split==5 or TREE["A_max"].iloc[int(childID)] ==A_max

            if split==1:
                subIntervalLength=(T1_max-T1_min)/nrChildren
                assert TREE["T1_min"].iloc[int(childID)]==T1_min+subIntervalLength*childNr
                assert TREE["T1_max"].iloc[int(childID)]==T1_min+subIntervalLength*childNr+subIntervalLength
            if split==2:
                subIntervalLength=(V1_max-V1_min)/nrChildren
                assert TREE["V1_min"].iloc[int(childID)]==V1_min+subIntervalLength*childNr
                assert TREE["V1_max"].iloc[int(childID)]==V1_min+subIntervalLength*childNr+subIntervalLength
            if split==3:
                subIntervalLength=(T2_max-T2_min)/nrChildren
                assert TREE["T2_min"].iloc[int(childID)]==T2_min+subIntervalLength*childNr
                assert TREE["T2_max"].iloc[int(childID)]==T2_min+subIntervalLength*childNr+subIntervalLength
            if split==4:
                subIntervalLength=(V2_max-V2_min)/nrChildren
                assert TREE["V2_min"].iloc[int(childID)]==V2_min+subIntervalLength*childNr
                assert TREE["V2_max"].iloc[int(childID)]==V2_min+subIntervalLength*childNr+subIntervalLength
            if split==5:
                subIntervalLength=(A_max-A_min)/nrChildren
                assert TREE["A_min"].iloc[int(childID)]==A_min+subIntervalLength*childNr
                assert TREE["A_max"].iloc[int(childID)]==A_min+subIntervalLength*childNr+subIntervalLength

    if split==6:
        ## all intervals will be split in half
        assert nrChildren==32

        ## we expect all 32 children to be stored in 32 consequtive rows of the TREE
        ## the first one should be at IDfirstChild
        ## we now iterate over all subintervals of the children, and verify their position in the solution tree
        
        childNr=0
        for a1 in range(2):
            for a2 in range(2):
                for a3 in range(2):
                    for a4 in range(2):
                        for a5 in range(2):
                            
                            childID=IDfirstChild+childNr
                            childNr+=1 
                            
                            if a1==0:
                                assert TREE["T1_min"].iloc[int(childID)]==T1_min
                                assert TREE["T1_max"].iloc[int(childID)]==(T1_min+T1_max)/2
                            else:
                                assert TREE["T1_min"].iloc[int(childID)]==(T1_min+T1_max)/2
                                assert TREE["T1_max"].iloc[int(childID)]==T1_max

                            if a2==0:
                                assert TREE["V1_min"].iloc[int(childID)]==V1_min
                                assert TREE["V1_max"].iloc[int(childID)]==(V1_min+V1_max)/2
                            else:
                                assert TREE["V1_min"].iloc[int(childID)]==(V1_min+V1_max)/2
                                assert TREE["V1_max"].iloc[int(childID)]==V1_max

                            if a3==0:
                                assert TREE["T2_min"].iloc[int(childID)]==T2_min
                                assert TREE["T2_max"].iloc[int(childID)]==(T2_min+T2_max)/2
                            else:
                                assert TREE["T2_min"].iloc[int(childID)]==(T2_min+T2_max)/2
                                assert TREE["T2_max"].iloc[int(childID)]==T2_max

                            if a4==0:
                                assert TREE["V2_min"].iloc[int(childID)]==V2_min
                                assert TREE["V2_max"].iloc[int(childID)]==(V2_min+V2_max)/2
                            else:
                                assert TREE["V2_min"].iloc[int(childID)]==(V2_min+V2_max)/2
                                assert TREE["V2_max"].iloc[int(childID)]==V2_max

                            if a5==0:
                                assert TREE["A_min"].iloc[int(childID)]==A_min
                                assert TREE["A_max"].iloc[int(childID)]==(A_min+A_max)/2
                            else:
                                assert TREE["A_min"].iloc[int(childID)]==(A_min+A_max)/2
                                assert TREE["A_max"].iloc[int(childID)]==A_max

# Checking the global Theorem

In [ ]:
for row in tqdm(TREE.itertuples(), total=len(TREE)): 
    if row.nodeType!= 1:
        continue
 
    wx = Integer(row.wx_nominator)
    wy = Integer(row.wy_nominator)
    den = Integer(row.w_denominator)

    w=matrix([[wx/den],[wy/den]])

    assert w.T*w==matrix([1]), f"The vector w not a unit vector, ID={row.ID} w={w}"

    T1_min=Integer(row.T1_min)/INTERVAL_DENOMINATOR
    T1_max=Integer(row.T1_max)/INTERVAL_DENOMINATOR
    V1_min=Integer(row.V1_min)/INTERVAL_DENOMINATOR
    V1_max=Integer(row.V1_max)/INTERVAL_DENOMINATOR
    T2_min=Integer(row.T2_min)/INTERVAL_DENOMINATOR
    T2_max=Integer(row.T2_max)/INTERVAL_DENOMINATOR
    V2_min=Integer(row.V2_min)/INTERVAL_DENOMINATOR
    V2_max=Integer(row.V2_max)/INTERVAL_DENOMINATOR
    A_min =Integer(row.A_min )/INTERVAL_DENOMINATOR
    A_max =Integer(row.A_max )/INTERVAL_DENOMINATOR

    t1=(T1_min+T1_max)/2
    v1=(V1_min+V1_max)/2
    t2=(T2_min+T2_max)/2
    v2=(V2_min+V2_max)/2
    a=(A_min+A_max)/2

    eps=max(T1_max-T1_min,V1_max-V1_min,T2_max-T2_min,V2_max-V2_min,A_max-A_min)/2
  
    S_=RatNOP[row.S_index,:].T ## S-tilde 
     
    G_=((R(a)*M(t1,v1)*S_).T*w-
    eps*abs_matrix((R_alpha_prime(a)*M(t1,v1)*S_).T*w)-
    eps*abs_matrix((R(a)*M_theta_prime(t1,v1)*S_).T*w)-
    eps*abs_matrix((R(a)*M_phi_prime(t1,v1)*S_).T*w)-
    9/2*eps**2-4*kappa*(1+3*eps))
  
    assert G_.dimensions()==(1,1), f"Something with G_ went terribly wrong"
    G_=G_[0,0]
    
    Term1=RatNOP*M(t2,v2).T*w
    Term2=RatNOP*M_theta_prime(t2,v2).T*w
    Term3=RatNOP*M_phi_prime(t2,v2).T*w
    
    HP=Term1+eps*abs_matrix(Term2)+eps*abs_matrix(Term3)
    
    H_=max(HP)[0]+2*eps**2+3*kappa*(1+2*eps)
    
    assert G_>H_, f"The global Theorem cannot be applied!"

print("Rational global Theorem can be applied for nodes with nodeType=1.")

# Checking the local Theorem

In [ ]:
#There exists an orthonormal matrix $L$ (i.e. $LL^T=Id$) such that $P_i=LQ_i$ for $i=1,2,3$.  
TREE_filtered = TREE[['P1_index', 'P2_index', 'P3_index', 'Q1_index', 'Q2_index', 'Q3_index']][TREE['nodeType'] == 2]
TREE_filtered = TREE_filtered.drop_duplicates()
TREE_filtered

ID = matrix(SR, 3,3)
ID[0,0]=1
ID[1,1]=1
ID[2,2]=1

for rowNr, row in tqdm(enumerate(TREE_filtered.itertuples(), start=1), total=len(TREE_filtered)):
    
    P1_index=row.P1_index
    P2_index=row.P2_index
    P3_index=row.P3_index
    Q1_index=row.Q1_index
    Q2_index=row.Q2_index
    Q3_index=row.Q3_index
  
    P1=NOP[P1_index,:].T
    P2=NOP[P2_index,:].T
    P3=NOP[P3_index,:].T
    Q1=NOP[Q1_index,:].T
    Q2=NOP[Q2_index,:].T
    Q3=NOP[Q3_index,:].T
    
    assert P1.T*P1==Q1.T*Q1
    assert P2.T*P2==Q2.T*Q2
    assert P3.T*P3==Q3.T*Q3
    
    assert P1.T*P2==Q1.T*Q2
    assert P1.T*P3==Q1.T*Q3
    assert P2.T*P3==Q2.T*Q3

    Q_matrix = matrix(SR, 3,3)
    Q_matrix[:,0]=Q1
    Q_matrix[:,1]=Q2
    Q_matrix[:,2]=Q3

    assert rank(Q_matrix)==3

In [ ]:
for row in tqdm(TREE.itertuples(), total=len(TREE), smoothing=float(1.0), mininterval=0.5):
    if row.nodeType!= 2:
        continue
    
    T1_min=Integer(row.T1_min)/INTERVAL_DENOMINATOR
    T1_max=Integer(row.T1_max)/INTERVAL_DENOMINATOR
    V1_min=Integer(row.V1_min)/INTERVAL_DENOMINATOR
    V1_max=Integer(row.V1_max)/INTERVAL_DENOMINATOR
    T2_min=Integer(row.T2_min)/INTERVAL_DENOMINATOR
    T2_max=Integer(row.T2_max)/INTERVAL_DENOMINATOR
    V2_min=Integer(row.V2_min)/INTERVAL_DENOMINATOR
    V2_max=Integer(row.V2_max)/INTERVAL_DENOMINATOR
    A_min =Integer(row.A_min )/INTERVAL_DENOMINATOR
    A_max =Integer(row.A_max )/INTERVAL_DENOMINATOR

    t1=(T1_min+T1_max)/2
    v1=(V1_min+V1_max)/2
    t2=(T2_min+T2_max)/2
    v2=(V2_min+V2_max)/2
    a= (A_min + A_max)/2

    eps=max(T1_max-T1_min,V1_max-V1_min,T2_max-T2_min,V2_max-V2_min,A_max-A_min)/2
  
    P1_index=row.P1_index
    P2_index=row.P2_index
    P3_index=row.P3_index
    Q1_index=row.Q1_index
    Q2_index=row.Q2_index
    Q3_index=row.Q3_index
  
    P1=RatNOP[P1_index,:].T
    P2=RatNOP[P2_index,:].T
    P3=RatNOP[P3_index,:].T
    Q1=RatNOP[Q1_index,:].T
    Q2=RatNOP[Q2_index,:].T
    Q3=RatNOP[Q3_index,:].T
    
    r=Integer(row.r)/R_DENOMINATOR 
    assert r>0, "r should be positive!"
    
    sigma_P=0
    sigma_Q=Integer(row.sigma_Q)
    assert sigma_Q==0 or sigma_Q==1, f"sigma_Q is neither 0 or 1"
    
    R_pi_2=matrix(QQ,[[0,-1],[1,0]]) ## is set to R(pi/2)
    
    X_1=X(t1,v1)
    X_2=X(t2,v2)
    
    M_t1_v1=M(t1,v1)
    M_t2_v2=M(t2,v2)

    r1=(R(a)*M_t1_v1*P1 - M_t2_v2*Q1)
    r2=(R(a)*M_t1_v1*P2 - M_t2_v2*Q2)
    r3=(R(a)*M_t1_v1*P3 - M_t2_v2*Q3)
    
    delta=3*kappa+max(sqrt_upper(NormSquared(r1)),
                      sqrt_upper(NormSquared(r2)),
                      sqrt_upper(NormSquared(r3)))/2
    
    g=2*eps*(142/100+eps)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ### (A_eps^Q): s_p condition ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    assert (-1)**sigma_P*ScalarProduct(X_1,P1)>142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    assert (-1)**sigma_P*ScalarProduct(X_1,P2)>142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    assert (-1)**sigma_P*ScalarProduct(X_1,P3)>142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ### (A_eps^Q): s_q condition ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  
    assert (-1)**sigma_Q*ScalarProduct(X_2,Q1)>142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    assert (-1)**sigma_Q*ScalarProduct(X_2,Q2)>142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    assert (-1)**sigma_Q*ScalarProduct(X_2,Q3)>142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
    ### Spanning Inequalities ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    assert ScalarProduct(R_pi_2*M_t1_v1*P1,M_t1_v1*P2)>g+6*kappa, f"Cannot apply rational local Theorem!"
    assert ScalarProduct(R_pi_2*M_t1_v1*P2,M_t1_v1*P3)>g+6*kappa, f"Cannot apply rational local Theorem!"
    assert ScalarProduct(R_pi_2*M_t1_v1*P3,M_t1_v1*P1)>g+6*kappa, f"Cannot apply rational local Theorem!"
    
    assert ScalarProduct(R_pi_2*M_t2_v2*Q1,M_t2_v2*Q2)>g+6*kappa, f"Cannot apply rational local Theorem!"
    assert ScalarProduct(R_pi_2*M_t2_v2*Q2,M_t2_v2*Q3)>g+6*kappa, f"Cannot apply rational local Theorem!"
    assert ScalarProduct(R_pi_2*M_t2_v2*Q3,M_t2_v2*Q1)>g+6*kappa, f"Cannot apply rational local Theorem!"

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ### Points are far from the origin ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    assert sqrt_lower(NormSquared(M_t2_v2*Q1))>= r+142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    assert sqrt_lower(NormSquared(M_t2_v2*Q2))>= r+142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    assert sqrt_lower(NormSquared(M_t2_v2*Q3))>= r+142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ### (B_eps^\Q) inequality ###
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    for i in range(1,4):
        if i==1:
            Qi_Index=Q1_index
            Qi=Q1
        if i==2:
            Qi_Index=Q2_index
            Qi=Q2
        if i==3:
            Qi_Index=Q3_index
            Qi=Q3
    
        denom1=sqrt_upper(NormSquared(M_t2_v2*Qi))+142/100*eps+3*kappa
    
        for A_index in range(RatNOP.nrows()):
            if A_index==Qi_Index:
                continue
                
            A=RatNOP[A_index,:].T

            nom1=ScalarProduct(M_t2_v2*Qi,M_t2_v2*Qi-M_t2_v2*A)
            nom2=10*kappa+2*eps*(sqrt_upper(NormSquared(Qi-A))+2*kappa)*(142/100+eps)
            
            nom=nom1-nom2

            denom2=sqrt_upper(NormSquared(M_t2_v2*(Qi-A)))+2*142/100*eps+6*kappa
      
            denom=denom1*denom2
      
            frac=nom/denom
      
            assert frac > (224/100*eps+delta)/r, f"Cannot apply rational local Theorem!"

print("Rational local Theorem can be applied for nodes with nodeType=2.")